In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
from link_prediction.model.link_predictor import Link_Prediction_Model
from link_prediction.my_util import my_utils

import torch

%matplotlib inline

current_dir = os.path.dirname(os.path.abspath(__file__))

In [ ]:
model = Link_Prediction_Model(dataset_name=args.dataset, val_ratio=0.05, test_ratio=0.1, data_dir=os.path.dirname(current_dir)+'/data')

In [ ]:
model(
    encode_modelname='GCNII', 
    decode_modelname='GAE',
    activation = 'relu', 
    self_loop_mask = True,
    num_hidden_channels = 256, 
    num_layers = 32,
    jk_mode = 'max',
    hidden_channels = None, 
    negative_injection=False,
    dropout = 0.5,
    sigmoid_bias = True,
    negative_sampling_ratio = 1,
    threshold = 0.5
)

optimizer = {}
optimizer['convs'] = torch.optim.Adam(model.encode_model.convs.parameters(), weight_decay=1e-10, lr=1e-1)
optimizer['lins'] = torch.optim.Adam(model.encode_model.lins.parameters(), weight_decay=1e-10, lr=1e-1)
optimizer['bias'] = torch.optim.Adam(model.decode_model.bias.parameters(), weight_decay=1e-3, lr=5e-2)
model.my_optimizer(optimizer)

scheduler = {}
scheduler['convs'] = torch.optim.lr_scheduler.StepLR(model.optimizer['convs'], step_size=2, gamma=0.5)
scheduler['lins'] = torch.optim.lr_scheduler.StepLR(model.optimizer['lins'], step_size=2, gamma=0.5)
scheduler['bias'] = torch.optim.lr_scheduler.ExponentialLR(model.optimizer['bias'], gamma=0.992)
model.my_scheduler(scheduler)

model.run_training(num_epochs=3000, print_log=False, save_dir=current_dir)
model.model_evaluate(validation=True, save=True)